# Text generator notebook from TF

This tutorial demonstrates how to generate text using a character-based RNN. You will work with a dataset of Shakespeare's writing from Andrej Karpathy's The Unreasonable Effectiveness of Recurrent Neural Networks. Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

- The model is character based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.
- The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.
- As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

In [2]:
import tensorflow as tf

import numpy as np
import os
import time

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


### Read the data

In [4]:
# Read then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# length of tect is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [5]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
# The unique characters in the file
vocab = sorted(set(text)) # ...cuz sets can allow only unique elements
print(f'{len(vocab)} unique characters')
print(vocab)

65 unique characters
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


### Process the text

The tf.keras.layers.StringLookup layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [7]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [8]:
# Now with the tf.keras.layers.StringLookup layer (vectorizer):
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [9]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use tf.keras.layers.StringLookup(..., invert=True).

In [10]:
# So this layer will be a decoder by only changing invert=True  
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [11]:
# lets check how the decoding went:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [12]:
# What if i pass in some random ints?
myints = tf.constant([31,32,46,60,12])

mychars = chars_from_ids(myints)
mychars

<tf.Tensor: shape=(5,), dtype=string, numpy=array([b'R', b'S', b'g', b'u', b';'], dtype=object)>

You can tf.strings.reduce_join to join the characters back into strings.

In [13]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [14]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### Create training examples and targets

In [15]:
# Read in all the text into tokenized form
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [16]:
# Create dataset 
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [17]:
# Checkout the decoded characters of first 10 sample token
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


The batch method lets you easily convert these individual characters to sequences of the desired size.

In [18]:
seq_length = 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings (using the previously defined text_from_ids - which joins the chars together):

In [19]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


For training you'll need a dataset of (input, label) pairs. Where input and label are sequences. At each time step the input is the current character and the label is the next character.

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [20]:
def split_input_target(sequence):
    input_text = sequence[:-1] # 
    target_text = sequence[1:]
    return input_text, target_text

sample = split_input_target("Sayonara Tako kun!")
sample

('Sayonara Tako kun', 'ayonara Tako kun!')

In [21]:
# Apply it on the whole, batched dataset
dataset = sequences.map(split_input_target)

In [22]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


### Create training batches

It is also a fastens things up if we use prefetch, and also recommended to use shuffle for efficient training:

In [23]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

### Build the model

In [24]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
# so each character will receive a 256 long embedding array of floats
# 256*65 total number
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [25]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim) # To convert input into embeddings
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True) # gru model
    self.dense = tf.keras.layers.Dense(vocab_size) # simple classif layer at the end

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [26]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

What happens inside the model:

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character.

In [27]:
# Lets run the model to see the shape of the output
# (currently untrained model...)
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [28]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Predictions for the 100 length input
example_batch_predictions[0]

<tf.Tensor: shape=(100, 66), dtype=float32, numpy=
array([[ 4.95469756e-03,  7.27180531e-03,  1.11259669e-02, ...,
         3.77608952e-03,  4.57991520e-03,  9.24432278e-03],
       [ 4.88151005e-03,  6.51077088e-03,  1.09781660e-02, ...,
         1.41227548e-03,  1.90219488e-02, -4.17208765e-03],
       [ 7.59055000e-03,  3.03673605e-03, -5.88342547e-04, ...,
        -7.74133345e-03,  3.46392533e-03,  3.63860303e-03],
       ...,
       [ 7.24139344e-03,  4.52696532e-03, -4.19180491e-04, ...,
        -9.41982027e-03,  8.46317969e-04,  3.12187197e-03],
       [ 3.66267562e-03,  2.37391400e-03, -5.16713597e-03, ...,
        -7.21296808e-03,  2.83708563e-03, -8.75522383e-03],
       [ 1.05778305e-02, -3.76375811e-03, -3.40450415e-03, ...,
        -7.07788998e-03, -9.20849852e-05,  8.58471380e-04]], dtype=float32)>

In [30]:
example_batch_cls_pred = tf.argmax(example_batch_predictions[0], axis=1)
example_batch_cls_pred

<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([40, 64, 11, 11, 43, 64, 25, 63, 29, 50, 37, 25, 27, 27, 27, 27,  5,
       65, 43, 64, 50, 10, 47, 43, 44, 28,  4, 11, 11, 10, 50, 22, 27, 46,
       50, 50, 50, 11, 34, 40,  0,  8, 34, 62, 56, 25, 34, 18, 63, 46, 50,
        5, 34, 21, 40, 27, 27, 43, 50, 64, 50, 50, 10, 25, 15, 33, 28, 35,
        7,  7, 16, 14, 11, 34, 41, 41, 45, 18, 43, 11, 11, 23, 40, 25, 40,
       40, 44, 50, 50, 50, 50, 11, 50, 50, 50, 18, 50, 11, 50, 23])>

In [31]:
# But as the notebook states we should do it like this:
# (Note: It is important to sample from this distribution as taking the argmax of the distribution can easily get the model stuck in a loop.)
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([10, 26, 52,  6,  6, 16, 15, 24, 19, 16, 21, 17,  5, 31,  3, 47,  2,
       60, 18, 26, 12, 34, 41, 54, 25,  0, 44,  9, 56, 34, 17, 50, 41, 28,
       30, 64, 23, 28, 29, 47, 26, 21, 32, 18, 32,  6,  4, 20,  4, 64, 39,
       47, 57, 37, 57, 52, 51, 43, 30, 56, 36, 45, 58, 24, 43, 53, 49, 51,
       58, 45, 61, 47, 16, 38, 39,  5, 31, 22, 45, 56, 52, 65, 57,  5, 14,
       34, 42, 57, 32, 51, 48, 43, 49, 51, 24, 48, 43, 35, 54,  3])

Decode these to see the text predicted by this untrained model:

In [32]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b' that dares love attempt;\nTherefore thy kinsmen are no let to me.\n\nJULIET:\nIf they do see thee, they'

Next Char Predictions:
 b"3Mm''CBKFCHD&R!h uEM;UboL[UNK]e.qUDkbOQyJOPhMHSES'$G$yZhrXrmldQqWfsKdnjlsfvhCYZ&RIfqmzr&AUcrSlidjlKidVo!"


### Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

#### Attach an optimizer, and a loss function

The standard tf.keras.losses.sparse_categorical_crossentropy loss function works in this case because it is applied across the last dimension of the predictions.
(bcuz the targets are label encoded...)

Because your model returns logits, you need to set the from_logits flag.

In [33]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [34]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.189888, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [35]:
tf.exp(example_batch_mean_loss).numpy()

66.015396

Configure the training procedure using the tf.keras.Model.compile method. Use tf.keras.optimizers.Adam with default arguments and the loss function.

In [36]:
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

### Configure checkpoints

Use a tf.keras.callbacks.ModelCheckpoint to ensure that checkpoints are saved during training:



In [37]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab set the runtime to GPU for faster training.

In [38]:
EPOCHS=20

In [45]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 15s 58ms/step - loss: 2.7283 - accuracy: 0.2807
Epoch 2/20
172/172 [==============================] - 11s 55ms/step - loss: 1.9961 - accuracy: 0.4161
Epoch 3/20
172/172 [==============================] - 12s 56ms/step - loss: 1.7195 - accuracy: 0.4905
Epoch 4/20
172/172 [==============================] - 11s 56ms/step - loss: 1.5557 - accuracy: 0.5335
Epoch 5/20
172/172 [==============================] - 11s 56ms/step - loss: 1.4560 - accuracy: 0.5593
Epoch 6/20
172/172 [==============================] - 12s 58ms/step - loss: 1.3875 - accuracy: 0.5761
Epoch 7/20
172/172 [==============================] - 12s 59ms/step - loss: 1.3337 - accuracy: 0.5893
Epoch 8/20
172/172 [==============================] - 13s 60ms/step - loss: 1.2897 - accuracy: 0.6007
Epoch 9/20
172/172 [==============================] - 13s 61ms/step - loss: 1.2490 - accuracy: 0.6111
Epoch 10/20
172/172 [==============================] - 12s 58ms/step - loss: 1.208

In [40]:
latest = tf.train.latest_checkpoint('/content/training_checkpoints/')
print(f'Latest model: {latest}')
model_loaded = model.load_weights(latest)

Latest model: /content/training_checkpoints/ckpt_20


### Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it. Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.

The following makes a single step prediciton.

In [41]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [42]:
# To generate next char
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [48]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
O, hust thou fox us roat,
Becours that offer me dear Jupitard,
That you shall cure it, subdue the Roman,
The prize of long past cure of this gentleman,
That you are yours, sleeps in't, and their loves and
remembrance will speak to their names. Johemia, some
conscience to a consoot, you do awmile me.

FRIAR LAURENCE:
Hast thou tail'd with the sleeping office?
Here, sir; as well used the realm; but they have cook'd
so fond to ambition. Their war,
When he shall find it meet and bay sent,--
Not arm'd on one that seestern such
A foolmand in this business.

POLIXENES:
How!
When good my life upon you! with my father, I do believe
Her father to my throne? but to make an end o'er the
eventness of a miserable bar, and blunts,
And bid them babies, compellons dalkners to the good am
I never talk; one that prover chaste,
For my past made gobed steel away,
Lord marshal, and Master not to field or strive,
Whilvest thou sleep, there's never yet sufficiently,
And givens every hand in falsehood o

The easiest thing you can do to improve the results is to train it for longer (try EPOCHS = 30).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text faster the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above.

In [43]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
for i in range(len(next_char)):
    print(result[i].numpy().decode('utf-8') + '\n' + '_'*80) # to print out the 5 inputs
print('\nRun time:', end - start)

ROMEO:
Shall I so much that poor infrunt, hair
Shaltestors are at Clarence be it consent.

RICHARD:
Ay, with thy lips: set thy brawl sits, and cry hath he should so:
But they shall speak from the cold conference?

QUEEN MARGARET:
So right have made a Jely, and thou, Heaven hath
Done no free to do it for ports.

First Senator:
No, misturn than this.

HENRY BOLINGBROKE:
Softly and to me? Both our hearts to find
Where our state were too much in your nameless, and you
Foul warrant him, a most disconfed enough;
'moughts are did mine: bid them I might show her mind: conceive
this oath I have any thing when stomands
That still-hould bid give you on: upon that time
It straits to hear a living him:
And I have leave in trisun through the king did scont
Of our devotion sworn like his opinion, where I give in
joy; but your uncle York and Lewis
besomity.

JULIET:
O, made not from the tomb;
A sight remembered and left lowd:
Enough for playing him,--do your love pussuned you.

AUTOLYCUS:
If they not,

### Export the generator

In [115]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')